In [2]:
import pandas as pd
import numpy as np
import glob
import cv2
from numpy import linalg as LA

import matplotlib.pyplot as plt
import itertools
import os
import random
import json
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn import preprocessing
from scipy import stats
from sklearn.model_selection import GridSearchCV, KFold
from math import sqrt
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss

In [3]:
directory = './regression_modeldata1'

with open('groundtruth.json') as json_file:
    gtdata = json.load(json_file)
subdirs = os.listdir(directory)

finalX = []
finalyLeft = []
finalyRight = []

for i in range(len(subdirs)):
    dir2pull = directory + '/' + subdirs[i] + '/'
    print(dir2pull)
    
    lefty = gtdata[subdirs[i]]['leftlength']
    righty = gtdata[subdirs[i]]['rightlength']
    
    
    for file in glob.glob(dir2pull + '*.csv'):
        data2read = file.replace("\\",'/')
        #print(data2read)
        singledata = pd.read_csv(data2read,header=None)
        singledata = np.array(singledata)
        
        dataarray = []
        
        for j in range(singledata.shape[1]):
            if isinstance(singledata[0,j],str):
                if singledata[0,j][0:2] == 'SI' or singledata[0,j][0:2]==' S':
                    datapoint = singledata[0,j].split('(')[1]
                elif singledata[0,j][0:2] == 'L ' or singledata[0,j][0:2] == 'R ':
                    datapoint = singledata[0,j][2:]
                else:
                    datapoint = singledata[0,j].split(')')[0]
                datapoint = float(datapoint)
            else:
                datapoint = singledata[0,j]
            dataarray.append(datapoint)
        procdata = np.array(dataarray)
        #print(procdata.shape)
        
        if procdata.shape[0] != 60:
            continue
            
        if procdata[6] == 0:
            continue
        
        finalX.append(procdata)
        finalyLeft.append(lefty + random.uniform(-0.5,0.5))
        finalyRight.append(righty + random.uniform(-0.5,0.5))
#print(len(finalX))

finalX = np.array(finalX)
print(finalX.shape)

finalyLeft = np.array(finalyLeft)
finalyRight = np.array(finalyRight)

X = finalX
#np.random.shuffle(X)
y = finalyRight
def shuffle_in_unison(a, b):
    n_elem = a.shape[0]
    indeces = np.random.choice(n_elem, size=n_elem, replace=False)
    return a[indeces], b[indeces]
    
X,y = shuffle_in_unison(X,y)

def shuffle_in_unison_three(a, b, c):
    n_elem = a.shape[0]
    indeces = np.random.choice(n_elem, size=n_elem, replace=False)
    return a[indeces], b[indeces], c[indeces]

X,yL,yR = shuffle_in_unison_three(X,finalyLeft,finalyRight)

./regression_modeldata1/sharma/
./regression_modeldata1/alex/
./regression_modeldata1/senthil/
./regression_modeldata1/vaneesh/
./regression_modeldata1/user1/
(84, 60)


In [8]:
featurenames = pd.read_csv('ModelDataTemplate.csv',header=None)
featurenames = featurenames.iloc[0,:]
print(featurenames)


# baseline -> left toe xyz = 100
# ground truth left = 27cm

# baseline -> new xyz = 90
# groundtruth != 24.3

0                     y1 baseline
1                     y2 baseline
2                     x1 baseline
3                     x2 baseline
4                     z1 baseline
5                     z2 baseline
6            distance from camera
7            left toe y cordinate
8            left toe x cordinate
9            left toe z cordinate
10               left foot length
11          right toe y cordinate
12          right toe x cordinate
13          right toe z cordinate
14              right foot length
15         camera grain intensity
16    transformation matrix (4x4)
17    transformation matrix (4x4)
18    transformation matrix (4x4)
19    transformation matrix (4x4)
20    transformation matrix (4x4)
21    transformation matrix (4x4)
22    transformation matrix (4x4)
23    transformation matrix (4x4)
24    transformation matrix (4x4)
25    transformation matrix (4x4)
26    transformation matrix (4x4)
27    transformation matrix (4x4)
28    transformation matrix (4x4)
29    transfor

In [4]:
#print(X[0,:])

# 27.5 cm.    /    26.6     =  22      /  x
# real_gt / translate_gt = real_ar / translate_ar


def dist_pt2line(linept1,linept2,toept):
    return LA.norm(np.cross(linept2-linept1, linept1-toept))/LA.norm(linept2-linept1)

def compute_translategt(real_gt):
    
    minlength = 0.23
    maxlength = 0.29
    adjust_pad = 0.004
    
    translate_gt = []
    
    current_length = real_gt
    
    while current_length > minlength:
        translate_gt.append(current_length - adjust_pad)
        current_length -= adjust_pad
    
    current_length = real_gt
    
    while current_length < maxlength:
        translate_gt.append(current_length + adjust_pad)
        current_length += adjust_pad
    
    return translate_gt
    
    
def compute_translatear(real_gt, translate_gt, real_ar):
    
    translate_ar = []
    
    for i in range(len(translate_gt)):
        translate_ar.append(real_ar * translate_gt[i] / real_gt)
    
    return translate_ar
    
    
def compute_new_xz(A,toept,translatear, realar, baselinept1=None,baselinept2=None):
    
    xzpts = []
    
    for i in range(len(translatear)):
        d2translate = realar - translatear[i]
        x,z = (toept[1],toept[2]) - d2translate*A
        
        xzpts.append([x,z])
        
        if baselinept1 is not None:
            print('index {}'.format(i))
            print(realar)
            print(translatear[i])
            print(d2translate)
            print(dist_pt2line(baselinept1,baselinept2,toept))
            print(dist_pt2line(baselinept1,baselinept2,(toept[0],x,z)))
        
    return xzpts
    
    


def generate_moredata(X2augment, yleft2augment, yright2augment):
    
    
    
    
    realleft_gt = yleft2augment / 1000
    realright_gt = yright2augment / 1000
    
    left_translate_gt = compute_translategt(realleft_gt)
    right_translate_gt = compute_translategt(realright_gt)
    
    left_real_ar = X2augment[10]
    right_real_ar = X2augment[14]
    
    #print(left_real_ar)
    
    
    left_translate_ar = compute_translatear(realleft_gt, left_translate_gt, left_real_ar)
    right_translate_ar = compute_translatear(realright_gt, right_translate_gt, right_real_ar)
    #print(left_translate_ar)
    
    
    baselinept1 = np.array((X2augment[0],X2augment[2],X2augment[4])) # y,x,z
    baselinept2 = np.array((X2augment[1],X2augment[3],X2augment[5]))
    
    lefttoept = np.array((X2augment[7],X2augment[8],X2augment[9]))
    righttoept = np.array((X2augment[11],X2augment[12],X2augment[13]))
    
    #print(dist_pt2line(baselinept1,baselinept2,lefttoept))
    
#     orig_dist_left = dist_pt2line(baselinept1,baselinept2,lefttoept)
#     orig_dist_right = dist_pt2line(baselinept1,baselinept2,righttoept)
    
#     print(orig_dist_left)
#     print('Original left length {}'.format(X2augment[10]))
    
    dx = baselinept2[1]-baselinept1[1]
    dy = baselinept2[2]-baselinept1[2]
    
    dist = np.sqrt(dx*dx + dy*dy)
    dx /= dist
    dy /= dist
    

    A = np.array((dy,-dx))
    
    #test = compute_new_xz(A,lefttoept,left_translate_ar, left_real_ar, baselinept1,baselinept2)
    
    left_xz_pts = compute_new_xz(A,lefttoept,left_translate_ar, left_real_ar)
    right_xz_pts = compute_new_xz(A,righttoept,right_translate_ar, right_real_ar)
    
    augmentedX = []
    augmentedyleft = []
    augmentedyright = []
    
    for i in range(len(left_translate_ar)):
        for j in range(len(right_translate_ar)):
            Xcopy = np.array(X2augment[:])
            #print(left_xz_pts)
            
            #print(Xcopy)
            
            Xcopy[8] = left_xz_pts[i][0]
            Xcopy[9] = left_xz_pts[i][1]
            Xcopy[10] = left_translate_ar[i]
            Xcopy[12] = right_xz_pts[j][0]
            Xcopy[13] = right_xz_pts[j][1]
            Xcopy[14] = right_translate_ar[j]
            
            augmentedX.append(Xcopy)
            
            #augmentedy.append([left_translate_gt[i],right_translate_gt[j]])
            augmentedyleft.append(left_translate_gt[i])
            augmentedyright.append(right_translate_gt[j])
    
    
    
#     plt.figure()
#     plt.scatter(baselinept1[1],baselinept1[2],c='r')
#     plt.scatter(baselinept2[1],baselinept2[2],c='r')
#     plt.scatter(newpoint[0],newpoint[1],c='b')
#     plt.scatter(lefttoept[1],lefttoept[2],c='g')
#     plt.show()
    
    
    return augmentedX, augmentedyleft, augmentedyright
#newX, newy = generate_moredata(X[0,:], yL[0], yR[0])
#print(np.array(newX)[:,10])

In [5]:
X_complete = []
y_complete_left = []
y_complete_right = []

for i in range(len(X)):
    newX, newyleft, newyright = generate_moredata(X[i,:],yL[i],yR[i])
    
    X_complete += newX
    y_complete_left += newyleft
    y_complete_right += newyright
    
    
print(np.array(X_complete).shape)
print(np.array(y_complete_left).shape)

(21504, 60)
(21504,)


In [6]:
# feature selection/removal

#X = X[:,0:15]

X = np.array(X_complete)
yL = np.array(y_complete_left) * 1000
yR = np.array(y_complete_right) * 1000

#X,yL,yR = shuffle_in_unison_three(X,yL,yR)

cv = KFold(n_splits=10,shuffle=True,random_state=234)
for train_index, test_index in cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    #y_train, y_test = yL[train_index], yL[test_index]
    y_train, y_test = yR[train_index], yR[test_index]
    
#     yL_train, yL_test = yL[train_index], yL[test_index]
#     yR_train, yR_test = yR[train_index], yR[test_index]
    
#     yL_train = np.expand_dims(yL_train,axis=1)
#     yR_train = np.expand_dims(yR_train,axis=1)
    
#     yL_test = np.expand_dims(yL_test,axis=1)
#     yR_test = np.expand_dims(yR_test,axis=1)
    
#     y_train = np.concatenate((yL_train,yR_train),axis=1)
#     y_test = np.concatenate((yL_test,yR_test),axis=1)                 
    
    # can comment out break and bring training into loop for cross validation
    break

print(X_train.shape)
print(X_test.shape)
    

# Now use scikit-learn's random forest regressor to train the model using
# the parameters pulled from above
#model = LinearRegression().fit(Xpreproc_train,y_train)
#model = RandomForestRegressor().fit(X_train,y_train)
model = GradientBoostingRegressor(n_estimators=200, max_depth=7).fit(X_train, y_train)
#model = GradientBoostingRegressor(n_estimators = 500, max_depth=5, 
#                                   min_samples_split = 3, min_samples_leaf=5).fit(X_train,y_train)

# submodel = GradientBoostingRegressor(n_estimators = 500, max_depth=10, 
#                                      min_samples_split = 5, min_samples_leaf=1)
# model = MultiOutputRegressor(submodel).fit(X_train,y_train)


#model = model.fit(X_train,y_train)

# validation metrics calculated here to examine model performance
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)


train_rmse = sqrt(mean_squared_error(y_train[:], ypred_train[:]))
test_rmse = sqrt(mean_squared_error(y_test[:],ypred_test[:]))
train_mae = mean_absolute_error(y_train[:], ypred_train[:])
test_mae = mean_absolute_error(y_test[:],ypred_test[:])

print('Train RMSE {}, Train MAE {}, Test RMSE {}, Test MAE {}'.format(train_rmse,train_mae,test_rmse,test_mae))


(19353, 60)
(2151, 60)
Train RMSE 0.19762220955575294, Train MAE 0.1468626660444538, Test RMSE 0.2615659431738249, Test MAE 0.16963584252474626


In [9]:
importance_values = model.feature_importances_
sort_importances = np.argsort(-importance_values)

for i in range(10):
    print('Top {} feature: {}, index {}'.format(i,featurenames.iloc[sort_importances[i]],sort_importances[i]))
    print('Percentage important: {}'.format(importance_values[sort_importances[i]]))

Top 0 feature: right foot length, index 14
Percentage important: 0.6130493762157019
Top 1 feature: camera grain intensity, index 15
Percentage important: 0.0644320775461067
Top 2 feature: distance from camera, index 6
Percentage important: 0.025319463124071954
Top 3 feature: transformation matrix (4x4), index 22
Percentage important: 0.024354188858420427
Top 4 feature: euler angle y , index 33
Percentage important: 0.024023452588134448
Top 5 feature: left toe y cordinate, index 7
Percentage important: 0.01913024471977998
Top 6 feature: transformation matrix (4x4), index 21
Percentage important: 0.015955502530322244
Top 7 feature: transformation matrix (4x4), index 20
Percentage important: 0.015914949571168964
Top 8 feature: euler angle x , index 32
Percentage important: 0.014829725200836289
Top 9 feature: left toe x cordinate, index 8
Percentage important: 0.014376177175806859


In [23]:
import coremltools

coreml_model = coremltools.converters.sklearn.convert(model)
coreml_model.save('ardata2length_right.mlmodel')